# Kotlin **Asynchronous/Concurrrent** Programming Basics

- **Coroutine scopes** and **launching** simple **coroutines** are discussed.

In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors
import kotlin.coroutines.CoroutineContext

## Coroutine Scopes

- Two fundamental **coroutine bilders** in Kotlin are `runBlocking` and `coroutineScope`.
- Both create **coroutine scopes** then we can run our coroutines in their scope.
- All codes in the defined scopes (either launched or direct alls) should complete running so that the scop finishes.
- `coroutineScope` builder returns a suspend function but `runBlocking` acts as normal function and blocks the assigned thread for itself.
- In a **coroutine scope**, code execution is structured as: **Run**->**Suspend/Finish?**->**Run Jobs**->**Repeat/Procceed?**->**Finish and Return**.

In [2]:
// Using 'coroutineScope' coroutine builder

/*              --------------- IMPORTANT ---------------
Intentionally, we limit the context to only 1 thread to see the non-blocking nature of coroutines.
Creating custom 'CoroutineContext' will be discussed in more detailes later. 
*/
val threadNo = 1
val context: CoroutineContext = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()
//              -----------------------------------------

// Defining the entry point with 'runBlocking'
fun main() = runBlocking(context) { // the defined  'context' limits execution to 1 thread.
    //.............runBlocking coroutine scope.................
    println("Program started")
    
    launch { // Lets launch a coroutine to see if the thread gets blocked by other suspend functions.
        println("The thread is not blocked. Just wait for resume ... ")
        delay(1000)
        println("The thread is not blocked; resumed ")
    }

    val result = somSuspendFun()
    println("Result code is: $result")
    
    println("Program finished")
}

// Creating a suspend function using 'coroutineScope' builder
// 'suspend' keyword is used here because the coroutine is not a blocking function but a suspending one.
suspend fun somSuspendFun(): Int = coroutineScope { // Only for experiment, change 'coroutineScope' to 'runBlocking' to see how execution gets blocked
    //.............somSuspendFun coroutine scope.................
    // This function isolates its execution scope from outer scopes.
    // This function has its own scope, but (as default,) inherits its context from the outer block, e.g runBlocking.
    // So it uses runBlocking context and its only thread.
 
    delay(5_000) // somSuspendFun will suspend for 5 seconds.

    // Let's also launch a coroutine in this scope.
    // REMEMBER: 'launch' can only launch coroutines in a coroutine scope.
    launch { // This will be executed after 'println("Hello World")' because launched jobs are queued.
        println("somSuspendFun finished")
    }

    println("Hello World")
    
    1 // The lambda retruns an Int which is the return value of 'somSuspendFun'
    // we can also use 'return@coroutineScope 1'
}

// IMPORTANT: somSuspendFun does not block the thread; when it gets suspended, other jobs (if any) get executed.
main()

/* 
VERY INPORTANT: We are responsible for the context lifecycle management
because we have defined a dedicated 'job' for it;
we must 'cancel()' it when we are done.
*/
context.cancel() 

Program started
The thread is not blocked. Just wait for resume ... 
The thread is not blocked; resumed 
Hello World
somSuspendFun finished
Result code is: 1
Program finished


## Launching coroutines by `launch/async`, returning a `Job/Deferred` objects and awaiting for the result

In [3]:
// Getting the 'Job' instance and joining

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started")

    // REMEMBER: 'launch' can only launch coroutines in a coroutine scope.
    // 'launch' queues the coroutine.
    // 'launch' coroutine builder returns a 'Job' instance
    val job: Job = launch { // 'launch' coroutine builder, returns a 'Job'. 
        println("Hello")
        // 'launch' returns 'Unit'
    }

    // It is possible to await for launch execution. 'join()' is a suspend function.
    // Code suspends here until the 'job' completes.
    job.join() // Comment this code to see the difference in execution order.
    
    println("Program finished")
}

main()

Program started
Hello
Program finished


In [4]:
// Getting the 'Deferred' instance and awaiting for the result

// The structure async/await is somehow similar to JavaScript async/await
fun main() = runBlocking {
    //.............runBlocking coroutine scope.................
    println("Program started")
    
    // 'async' is similar to 'launch' with the difference that it can return a value.
    // A differed is a generic 'Deferred<[Return Type]>' type. Generics will be covered later.
    val deferred1: Deferred<String> = async {
        delay(1000L)
        "Job 1 result"
    }

    val deferred2: Deferred<String> = async {
        delay(500L)
        "Job 2 result"
    }

    // Await the results of both deferred.
    println("Waiting for both jobs to finish...")
    val result1 = deferred1.await()  // Wait for the first job to complete and get the result
    val result2 = deferred2.await()  // Wait for the second job to complete and get the result

    // Print the results
    println("Job 1 finished with result: $result1")
    println("Job 2 finished with result: $result2")

    println("Program finished")
}

main()

Program started
Waiting for both jobs to finish...
Job 1 finished with result: Job 1 result
Job 2 finished with result: Job 2 result
Program finished


In [5]:
// Using 'awaitAll'

fun main() = runBlocking {
    //.............runBlocking coroutine scope.................
    println("Program started")
    
    val deferred1 = async {
        delay(1000L)
        "Job 1 result"
    }

    val deferred2 = async {
        delay(500L)
        "Job 2 result"
    }

    // Await the results of both jobs using awaitAll()
    println("Waiting for both jobs to finish...")
    val results = awaitAll(deferred1, deferred2)  // Wait for both jobs to complete

    // Print the results
    println("Job 1 finished with result: ${results[0]}")
    println("Job 2 finished with result: ${results[1]}")

    println("Program finished")
}

main()

Program started
Waiting for both jobs to finish...
Job 1 finished with result: Job 1 result
Job 2 finished with result: Job 2 result
Program finished


In [6]:
// 'joinAll' is also available for 'Job's

fun main() = runBlocking {
    //.............runBlocking coroutine scope.................
    println("Program started")

    // Declare job1 and job2 as Job variables
    val job1 = launch {
        delay(1000L)
        println("Job 1 finished after 1 second")
    }

    val job2 = launch {
        delay(500L)
        println("Job 2 finished after 0.5 seconds")
    }

    // Await for all launched jobs to finish using joinAll()
    println("Waiting for both jobs to finish...")
    joinAll(job1, job2)  // Wait for all jobs to complete

    println("Program finished")
}

main()

Program started
Waiting for both jobs to finish...
Job 2 finished after 0.5 seconds
Job 1 finished after 1 second
Program finished
